In [19]:
from typing import Optional, Type
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import BaseTool

class MultiplierInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

class CustomMultiplierTool(BaseTool):
    name = "Calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = MultiplierInput

    def _run(
        self, a: int, b: int
    ) -> str:
        """Use the tool."""
        return a * b

    async def _arun(
        self,
        a: int,
        b: int,
    ) -> str:
        """Use the tool asynchronously."""
        return self._run(a,b)

In [26]:
from langchain.agents import AgentExecutor, create_openai_tools_agent,Tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
from langchain.globals import set_verbose
import dotenv

dotenv.load_dotenv()

def create_agent(model_name):

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    tools = [CustomMultiplierTool()]
    
    llm = ChatOpenAI(model=model_name, temperature=0)

    system_message = """You are a general AI assistant.
    You have access to several tool. Don't answer the question if you are not getting the answer from a tool.
    Don't put safety and cultural warnings. Only warn about security. 
    """

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_message),
            MessagesPlaceholder("chat_history", optional=True),
            ("human", "{input}"),
            MessagesPlaceholder("agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    agent_exe = AgentExecutor(agent=agent, tools=tools,memory=memory)
    return agent_exe

async def run_agent(agent,user_query):
    #print(agent.memory.chat_memory.messages[-2:] if len(agent.memory.chat_memory.messages) > 1 else "")
    #set_verbose(True)
    # print(agent.memory.chat_memory)
    print('********************')
    # print()
    return await agent.ainvoke(input={"input":user_query},verbose=True)

In [29]:
agent_exe = create_agent("gpt-3.5-turbo")

In [30]:
await run_agent(agent_exe,"2 multiply by 3")

********************


> Entering new AgentExecutor chain...

Invoking: `Calculator` with `{'a': 2, 'b': 3}`


62 multiplied by 3 is equal to 6.

> Finished chain.


{'input': '2 multiply by 3',
 'chat_history': [HumanMessage(content='2 multiply by 3'),
  AIMessage(content='2 multiplied by 3 is equal to 6.')],
 'output': '2 multiplied by 3 is equal to 6.'}